# Сесија 5 – Оркестратор за више агената

Приказује једноставан процес са два агента (Истраживач -> Уредник) користећи Foundry Local.


### Објашњење: Инсталација зависности
Инсталира `foundry-local-sdk` и `openai` који су потребни за приступ локалном моделу и завршетке разговора. Идемпотентно.


# Сценарио
Имплементира минимални оркестраторски образац са два агента:
- **Агент истраживач** прикупља сажете чињеничне информације
- **Агент уредник** преписује за јасноћу на извршном нивоу

Демонстрира заједничку меморију по агенту, секвенцијално прослеђивање међурезултата и једноставну функцију цевовода. Може се проширити на више улога (нпр. Критичар, Проверавач) или паралелне гране.

**Променљиве окружења:**
- `FOUNDRY_LOCAL_ALIAS` - Подразумевани модел за коришћење (подразумевано: phi-4-mini)
- `AGENT_MODEL_PRIMARY` - Примарни модел агента (надјачава ALIAS)
- `AGENT_MODEL_EDITOR` - Модел агента уредника (подразумевано: примарни)

**Референца за SDK:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**Како функционише:**
1. **FoundryLocalManager** аутоматски покреће Foundry Local сервис
2. Преузима и учитава наведени модел (или користи кеширану верзију)
3. Обезбеђује OpenAI-компатибилан крајњи тачку за интеракцију
4. Сваки агент може користити различит модел за специјализоване задатке
5. Уграђена логика поновног покушаја елегантно решава привремене грешке

**Кључне карактеристике:**
- ✅ Аутоматско откривање и иницијализација сервиса
- ✅ Управљање животним циклусом модела (преузимање, кеширање, учитавање)
- ✅ OpenAI SDK компатибилност за познат API
- ✅ Подршка за више модела ради специјализације агената
- ✅ Робусно руковање грешкама са логиком поновног покушаја
- ✅ Локална инференција (није потребан cloud API)


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### Објашњење: Основни увози и типизација
Уводи dataclasses за складиштење порука агента и типске наговештаје ради јасноће. Увози Foundry Local менаџер + OpenAI клијент за наредне акције агента.


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### Објашњење: Иницијација модела (SDK образац)
Користи Foundry Local Python SDK за поуздано управљање моделима:
- **FoundryLocalManager(alias)** - Аутоматски покреће сервис и учитава модел преко алијаса
- **get_model_info(alias)** - Разрешава алијас у конкретан ID модела
- **manager.endpoint** - Обезбеђује крајњу тачку сервиса за OpenAI клијента
- **manager.api_key** - Обезбеђује API кључ (опционо за локалну употребу)
- Подржава одвојене моделе за различите агенте (примарни наспрам уредника)
- Уграђена логика поновног покушаја са експоненцијалним одлагањем за већу отпорност
- Провера везе како би се осигурало да је сервис спреман

**Кључни SDK образац:**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**Управљање животним циклусом:**
- Менаџери се чувају глобално ради правилног чишћења
- Сваки агент може користити различит модел за специјализацију
- Аутоматско откривање сервиса и руковање везом
- Глатко поновно покушавање са експоненцијалним одлагањем у случају неуспеха

Ово обезбеђује правилну иницијацију пре него што оркестрација агента почне.

**Референца:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### Објашњење: Класе Agent и Memory
Дефинише лагани `AgentMsg` за меморијске уносе и `Agent` који обухвата:
- **Улога система** - Личност агента и упутства
- **Историја порука** - Одржава контекст разговора
- **Метод act()** - Извршава акције уз одговарајуће руковање грешкама

Агент може користити различите моделе (примарни наспрам уредника) и одржава изолован контекст за сваког агента. Овај образац омогућава:
- Перзистенцију меморије током акција
- Флексибилно додељивање модела по агенту
- Изолацију грешака и опоравак
- Лако повезивање и оркестрацију


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### Објашњење: Оркестрирани Пипелин
Креира два специјализована агента:
- **Истраживач**: Користи примарни модел, прикупља чињеничне информације
- **Уредник**: Може користити посебан модел (ако је конфигурисано), усавршава и преписује

Функција `pipeline`:
1. Истраживач прикупља сирове информације
2. Уредник их усавршава у излаз спреман за извршне одлуке
3. Враћа и посредне и коначне резултате

Овај образац омогућава:
- Специјализацију модела (различити модели за различите улоге)
- Побољшање квалитета кроз обраду у више фаза
- Праћење трансформације информација
- Лако проширење на више агената или паралелну обраду


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### Објашњење: Извршење и резултати цевовода
Извршава мулти-агентски цевовод на питање са темом усклађености + кашњења како би се демонстрирало:
- Вишестепена трансформација информација
- Специјализација агената и сарадња
- Побољшање квалитета излазних резултата кроз прераду
- Трасабилност (очување и посредних и коначних резултата)

**Структура резултата:**
- `question` - Оригинални упит корисника
- `research` - Сирови резултати истраживања (чињенични подаци)
- `final` - Прерађени извршни резиме
- `models` - Који модели су коришћени за сваку фазу

**Идеје за проширење:**
1. Додати агента критичара за преглед квалитета
2. Имплементирати паралелне истраживачке агенте за различите аспекте
3. Додати агента верификатора за проверу чињеница
4. Користити различите моделе за различите нивое сложености
5. Имплементирати повратне петље за итеративно побољшање


### Напредно: Прилагођавање конфигурације агента

Покушајте да прилагодите понашање агента модификовањем променљивих окружења пре покретања ћелије за иницијализацију:

**Доступни модели:**
- Користите `foundry model ls` у терминалу да видите све доступне моделе
- Примери: phi-4-mini, phi-3.5-mini, qwen2.5-7b, llama-3.2-3b, итд.


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**Одрицање од одговорности**:  
Овај документ је преведен помоћу услуге за превођење уз помоћ вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако се трудимо да обезбедимо тачност, молимо вас да имате у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати ауторитативним извором. За критичне информације препоручује се професионални превод од стране људи. Не сносимо одговорност за било каква погрешна тумачења или неспоразуме који могу произаћи из коришћења овог превода.
